In [1]:
import torch
import pandas as pd
import pysnooper
from transformers import BertTokenizer
from IPython.display import clear_output
from opencc import OpenCC

PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.4.0


In [2]:
vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 21128


In [3]:
data = []
with open('data/train.txt', 'r', encoding="utf-8") as fp:
    # remove first line
    line = fp.readline()
    line = fp.readline()
    need = 6000
    cnt = 1
    while line and cnt <= need:
        data.append(line)
        line = fp.readline()
        cnt += 1

In [4]:
data[0].split('_!_')

['1',
 '9648ddb62b2544c3bc12d7c9fed2a854',
 '周六晚到卖场听夜场摇滚',
 '中央路红星开启“爱木”狂购  三伏天已到，比天气更热的是红星美凯龙的第三届鲁班文化节。这个周末，想避暑的，想找童趣的，想听摇滚的，想买家具的，都可以去红星美凯龙中央路商场。  7月18日18:00-21:00，中央路红星爱木联盟的“仲夏夜party”将闪亮登场。既然是派对，当然少不了音乐，届时将有南京知名摇滚乐队现场驻唱，high唱“红苹果”国际潮流音乐。想抢购家具的消费者，可以通过本报专线025-*******提前预约夜购狂欢。凡成功预约报名的，可在活动签到现场获得夜购预约卡集，凭卡集可领取莫斯利安酸奶一箱或价值150元的丁渤验房券1张，礼品数量有限，领完即止。同时，业主凭卡集还可到夜场活动展厅，尊享百款夜购爆款商品。  作为本次国际潮流音乐之夜的全程赞助商，知名板式品牌红苹果联合红星推出“双红惠”活动，回报顾客。7月6日-19日，顾客关注“红苹果家具”微信公众号可抢红包，另外还可享全屋定制家具优惠，更多活动详情可见中央路红星店堂公告。  暑期活动怎能少得了孩子？近日，儿童家具品牌酷漫居进入中央路红星，7月18日-19日10:20-17:00，儿童t台秀、芭蕾舞表演、小丑游戏、儿童分房讲座等活动精彩纷呈。活动期间还另设儿童活动区域，钓鱼、橡皮泥手工、决明子沙池等着小朋友来玩耍。参与活动的小朋友签到还有精美学习套装赠送，数量有限，送完为止。  这个双休日，到中央路红星买家具的大朋友也有惊喜哦！成交价基础上最高直降5%。此外，会员购物积分将前所未有地直接翻10倍！凭会员卡及当日满额订单扣减积分还可兑换刮刮卡，冷藏箱、塔式电风扇、精美脸谱u盘、莫斯利安酸奶、五常大米、咖啡饮品券、非转基因食用油、书籍……100%有奖，3000份好礼等你抢。陈燕飞\n']

In [28]:
len(data)

5355

In [29]:
cc = OpenCC('s2t')

converted = cc.convert(data[1].split('_!_')[2])

In [30]:
label = []
title = []
content = []

for i in data:
    splited_data = i.split('_!_')
    label.append(splited_data[0])
    title.append(cc.convert(splited_data[2]))
    content.append(cc.convert(splited_data[3]))

In [31]:
df = pd.DataFrame({'title': title, 'content': content, 'label': label})

In [32]:
## label 0 for positive, 1 for neutural, 2 for negative
df.head()

,title,content,label
0,週六晚到賣場聽夜場搖滾,中央路紅星開啓“愛木”狂購 三伏天已到，比天氣更熱的是紅星美凱龍的第三屆魯班文化節。這個週...,1
1,北京老教授泄露，持有山河藥輔節後下跌公告，速速看看！！！,大家好，我是老王，一個地地道道的老股民 ，有過巔峯，也有過低谷，最高時期單月盈利近三百萬 ，...,1
2,張灘鎮積極開展基幹民兵訓練活動,（漢濱網訊）爲了加強基幹民兵隊伍建設，不斷提升民兵隊伍的國防意識和軍事技能。連日來，張灘鎮按...,0
3,倆小夥無證騎摩托，未成年還試圖闖卡！,兩輛摩托車被正在進行交通違法查處的交警攔了下來。其中一輛摩托車的駕駛人還試圖闖卡逃跑，他們又...,2
4,不好意思，你不配做深圳人!_搜狐汽車_搜狐網,兄dei，路怒+地圖炮一起服用，等待你的就不只是交警的行政處罰，還有無數想人肉你的惠州老鐵。...,2


In [36]:
df.to_csv('data/train_tc.csv')
print(len(df))
print(df.label.value_counts() / len(df))

5355
1    0.498973
2    0.393091
0    0.107937
Name: label, dtype: float64


In [6]:
"""
此 Dataset 每次將 csv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""

from torch.utils.data import Dataset

class sentimentDataset(Dataset):
    # 讀取處理後的 csv 並初始化參數
    def __init__(self, mode, tokenizer, max_len=512):
        assert mode in ['train', 'valid', 'test']
        self.mode = mode
        self.df = pd.read_csv('data/' + mode + '_tc.csv')
        self.len = len(self.df)
        self.tokenizer = tokenizer
        self.max_len = max_len
        
#     @pysnooper.snoop()  # 加入以了解所有轉換過程
    def __getitem__(self, idx):
        if self.mode == 'train' or self.mode == 'valid':
            content_id, title, content, label = self.df.iloc[idx, :].values
            label_tensor = torch.tensor(label)
            
        # 建立 title 的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ['[CLS]']
        tokens_title = self.tokenizer.tokenize(title)
        word_pieces += tokens_title + ['[SEP]']
        len_title = len(word_pieces)
        
        # 建立 content 的 BERT tokens 並加入分隔符號 [SEP]
        tokens_content = self.tokenizer.tokenize(content)
        word_pieces += tokens_content + ['[SEP]']
        if len(word_pieces) > self.max_len:
            word_pieces = word_pieces[:self.max_len]
        len_content = len(word_pieces) - len_title
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將 tiel 包含 [SEP] 的 token 位置設為 0，其他為 1 表示 content
        segments_tensor = torch.tensor([0] * len_title + [1] * len_content, dtype=torch.long)
        
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len

In [7]:
trainset = sentimentDataset('train', tokenizer=tokenizer, max_len=200)

In [10]:
# 選擇第一個樣本
sample_idx = 0

# 將原始文本拿出做比較
content_id, title, content, label = trainset.df.iloc[sample_idx, :].values

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子 1：{title}
句子 2：{content}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

label_tensor   ：{label_tensor}

--------------------

[還原 tokens_tensors]
{combined_text}
""")

[原始文本]
句子 1：週六晚到賣場聽夜場搖滾
句子 2：中央路紅星開啓“愛木”狂購  三伏天已到，比天氣更熱的是紅星美凱龍的第三屆魯班文化節。這個週末，想避暑的，想找童趣的，想聽搖滾的，想買傢俱的，都可以去紅星美凱龍中央路商場。  7月18日18:00-21:00，中央路紅星愛木聯盟的“仲夏夜party”將閃亮登場。既然是派對，當然少不了音樂，屆時將有南京知名搖滾樂隊現場駐唱，high唱“紅蘋果”國際潮流音樂。想搶購傢俱的消費者，可以通過本報專線025-*******提前預約夜購狂歡。凡成功預約報名的，可在活動簽到現場獲得夜購預約卡集，憑卡集可領取莫斯利安酸奶一箱或價值150元的丁渤驗房券1張，禮品數量有限，領完即止。同時，業主憑卡集還可到夜場活動展廳，尊享百款夜購爆款商品。  作爲本次國際潮流音樂之夜的全程贊助商，知名板式品牌紅蘋果聯合紅星推出“雙紅惠”活動，回報顧客。7月6日-19日，顧客關注“紅蘋果傢俱”微信公衆號可搶紅包，另外還可享全屋定製傢俱優惠，更多活動詳情可見中央路紅星店堂公告。  暑期活動怎能少得了孩子？近日，兒童傢俱品牌酷漫居進入中央路紅星，7月18日-19日10:20-17:00，兒童t臺秀、芭蕾舞表演、小丑遊戲、兒童分房講座等活動精彩紛呈。活動期間還另設兒童活動區域，釣魚、橡皮泥手工、決明子沙池等着小朋友來玩耍。參與活動的小朋友簽到還有精美學習套裝贈送，數量有限，送完爲止。  這個雙休日，到中央路紅星買傢俱的大朋友也有驚喜哦！成交價基礎上最高直降5%。此外，會員購物積分將前所未有地直接翻10倍！憑會員卡及當日滿額訂單扣減積分還可兌換刮刮卡，冷藏箱、塔式電風扇、精美臉譜u盤、莫斯利安酸奶、五常大米、咖啡飲品券、非轉基因食用油、書籍……100%有獎，3000份好禮等你搶。陳燕飛

分類  ：1

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([  101,  6867,  1063,  3241,  1168,  6546,  1842,  5481,  1915,  1842,
         3015,  4020,   102,   704,  1925,  6662,  5148,  3215,  7274,  1559,
          100

In [8]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `sentimentDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `sentimentDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生 masks_tensors

def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        label = torch.stack([s[2] for s in samples])
    else:
        label = None
        
    # zero pad 到同一長度
    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label

In [9]:
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 5
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

In [13]:
data = next(iter(trainloader))

tokens_tensors, segments_tensors, masks_tensors, label = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label.shape}
{label}
""")


tokens_tensors.shape   = torch.Size([20, 200]) 
tensor([[ 101, 6867, 1063,  ...,  115,  115,  115],
        [ 101, 1266,  776,  ...,  671, 7547, 8024],
        [ 101, 2484, 4121,  ..., 4708, 3146, 7968],
        ...,
        [ 101, 6043, 7943,  ..., 2972, 4708,  671],
        [ 101,  679, 4500,  ..., 3621, 2130, 1059],
        [ 101, 2255, 6205,  ...,    0,    0,    0]])
------------------------
segments_tensors.shape = torch.Size([20, 200])
tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0]])
------------------------
masks_tensors.shape    = torch.Size([20, 200])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])
------------------------
label_ids.shape     

In [10]:
# 載入一個可以做中文多分類任務的模型，n_class = 3
from transformers import BertForSequenceClassification

PRETRAINED_MODEL_NAME = 'bert-base-chinese'
NUM_LABELS = 3

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=3, bias=True)


In [11]:
"""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
    
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將 tenosrs 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, token_type_ids=segments_tensors, attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 紀錄當前 batch
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
                
        if compute_acc:
            acc = correct / total
            return predictions, acc
        return predictions

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print('device:', device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)

device: cuda:0


In [13]:
print('acc:',acc)

acc: 0.1161531279178338


In [14]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
     
model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")


整個分類模型的參數量：102269955
線性分類器的參數量：2307



In [15]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optmizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

EPOCHS = 3

for epoch in range(EPOCHS):
    running_loss = 0.0
    
    for data in trainloader:
        
        tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]
        
        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors,
                        token_type_ids=segments_tensors,
                        attention_mask=masks_tensors,
                        labels=labels)
        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()
        
        # 紀錄當前 batch loss
        running_loss += loss.item()
        
    # 計算準確率
    _, acc = get_predictions(model, trainloader, compute_acc=True)
    
    print('[epoch %d] loss: %.3f, acc: %.3f' % (epoch+1, running_loss, acc))

[epoch 1] loss: 532.013, acc: 0.894
[epoch 2] loss: 304.207, acc: 0.940
[epoch 3] loss: 183.202, acc: 0.943
Wall time: 25min 7s


In [16]:
torch.save(model.state_dict(), 'data/epoch_3_batch_5_all_samples')

In [32]:
PRETRAINED_MODEL_NAME = 'bert-base-chinese'
NUM_LABELS = 3

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

model.load_state_dict(torch.load('data/epoch_3_batch_5_1000_samples'))

<All keys matched successfully>

In [33]:
trainset.df.label.value_counts() / len(trainset.df)

1    0.498973
2    0.393091
0    0.107937
Name: label, dtype: float64

In [34]:
validset = sentimentDataset('valid', tokenizer=tokenizer, max_len=200)

BATCH_SIZE = 20
validloader = DataLoader(validset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

In [35]:
%%time
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print('device:', device)
model = model.to(device)
result, acc = get_predictions(model, validloader, compute_acc=True)

device: cuda:0
Wall time: 22.7 s


In [36]:
print(acc)

0.845691382765531


In [15]:
validset.df.label.value_counts() / len(validset.df)

1    0.492986
2    0.420842
0    0.086172
Name: label, dtype: float64

In [37]:
result = result.tolist()

In [38]:
origin = validset.df.label.tolist()

In [39]:
# acc 0.845691382765531
print('data distribution for valid data')
print(f'0\t{origin.count(0)}')
print(f'1\t{origin.count(1)}')
print(f'2\t{origin.count(2)}')
cnt = 0
print('true positive for epoch_3_batch_5_1000_samples model:')
for i in range(3):
    cnt = 0
    for o, p in zip(origin, result):
        if o==i and p==i:
            cnt += 1
    print(f'{i}\t{cnt}')

data distribution for valid data
0	86
1	492
2	420
true positive for epoch_3_batch_5_1000_samples model:
0	42
1	437
2	365


In [25]:
# acc 0.8066132264529058
print('data distribution for valid data')
print(f'0\t{origin.count(0)}')
print(f'1\t{origin.count(1)}')
print(f'2\t{origin.count(2)}')
cnt = 0
print('true positive for epoch_3_batch_5_all_samples model:')
for i in range(3):
    cnt = 0
    for o, p in zip(origin, result):
        if o==i and p==i:
            cnt += 1
    print(f'{i}\t{cnt}')

data distribution for valid data
0	86
1	492
2	420
true positive for epoch_3_batch_5_all_samples model:
0	67
1	381
2	357
